In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import pymongo
from datetime import datetime
import pytz

from telegram import InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import ApplicationBuilder
from telegram.constants import ParseMode

from bot import config
from bot import database
from bot.app import show_balance_handle

In [ ]:
application = (
    ApplicationBuilder()
    .token(config.telegram_token)
    .build()
)

db = database.Database()

In [ ]:
newsletter_id = "token_balance_0"

db.create_newsletter(newsletter_id)
already_sent_to_user_ids = set(db.get_newsletter_attribute(newsletter_id, "already_sent_to_user_ids"))

print(f"Already sent to users: {already_sent_to_user_ids}")

In [ ]:
user_dicts = list(db.user_collection.find({"token_balance": 0}))
print(f"Found {len(user_dicts)} users")

In [ ]:
for user_dict in user_dicts:
    if user_dict["token_balance"] > 0:
        print(f"Skipping {user_dict['_id']}. Token balance > 0")
        continue
        
    if user_dict["_id"] in already_sent_to_user_ids:
        print(f"Skipping {user_dict['_id']}. Already sent before")
        continue
    
    text = f"🔴 You have <b>NO</b> tokens left\n"
    text += f"You totally spent <b>{user_dict['n_used_tokens']}</b> tokens\n"

    text += f"\nTop up your balance with <b>crypto</b> by choosing an option below. If crypto is not your thing, write me  and we will figure something out."
    reply_markup = InlineKeyboardMarkup([
        [InlineKeyboardButton("🟣 +100K tokens – 2.99$", callback_data=f"send_payment|2.99|100000")],
        [InlineKeyboardButton("🟣 +500K tokens – 9.99$", callback_data=f"send_payment|9.99|500000")],
        [InlineKeyboardButton("🟣 +1M tokens – 16.99$", callback_data=f"send_payment|16.99|1000000")]
    ])
    
    try:
        await application.bot.send_message(
            user_dict['chat_id'],
            text,
            reply_markup=reply_markup,
            parse_mode=ParseMode.HTML
        )
        print(f"Successfully sent to {user_dict['_id']}")
              
        db.add_user_to_newsletter(newsletter_id, user_dict["_id"])
    except Exception as e:
        print(f"Failed to send message to {user_dict['_id']}. Reason: {e}")